In [1]:
!pip install mediapipe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 7.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 MB 1.7 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.6/477.6 kB 11.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 3.9 MB/s eta 0:00:0000:0100:01m


In [1]:
import cv2
import mediapipe as mp

def main():
    # Initialize MediaPipe Hand model
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # Initialize OpenCV VideoCapture
    
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        # Read frame from camera
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    # Get the coordinates of each finger's joints
                    finger_coordinates = {
                        "thumb": [
                            hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC],
                            hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
                        ],
                        "index": [
                            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
                        ],
                        "middle": [
                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
                        ],
                        "ring": [
                            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
                        ],
                        "pinky": [
                            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]
                        ]
                    }
                    
                    # Convert normalized coordinates to pixel coordinates
                    height, width, _ = frame.shape
                    for finger, joints in finger_coordinates.items():
                        for joint in joints:
                            cx, cy = int(joint.x * width), int(joint.y * height)
                            joint.x = cx
                            joint.y = cy
                    
                    # Draw circles at the coordinates
                    for finger, joints in finger_coordinates.items():
                        for joint in joints:
                            cv2.circle(frame, (int(joint.x), int(joint.y)), 5, (0, 255, 0), cv2.FILLED)
        
        # Display the frame
        cv2.imshow('Hand Tracking', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


I0000 00:00:1715148009.923487    6849 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715148009.926217    6918 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.5-1ubuntu1), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 17.0.6, DRM 3.57, 6.8.0-31-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/home/deadbytes/Documents/ML/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type


In [2]:
import psutil

def get_running_apps():
    """
    Get a list of running applications.

    Returns:
        list: A list of running application names.
    """
    running_apps = []
    for proc in psutil.process_iter(['name']):
        if proc.info['name']:
            running_apps.append(proc.info['name'])
    return running_apps

# Example usage
running_apps = get_running_apps()
print("Running Applications:")
for app in running_apps:
    print(app)


Running Applications:
systemd
kthreadd
pool_workqueue_release
kworker/R-rcu_g
kworker/R-rcu_p
kworker/R-slub_
kworker/R-netns
kworker/0:0H-events_highpri
kworker/R-mm_pe
rcu_tasks_kthread
rcu_tasks_rude_kthread
rcu_tasks_trace_kthread
ksoftirqd/0
rcu_preempt
migration/0
idle_inject/0
cpuhp/0
cpuhp/2
idle_inject/2
migration/2
ksoftirqd/2
kworker/2:0H-events_highpri
cpuhp/4
idle_inject/4
migration/4
ksoftirqd/4
kworker/4:0H-events_highpri
cpuhp/6
idle_inject/6
migration/6
ksoftirqd/6
kworker/6:0H-events_highpri
cpuhp/8
idle_inject/8
migration/8
ksoftirqd/8
kworker/8:0H-events_highpri
cpuhp/10
idle_inject/10
migration/10
ksoftirqd/10
kworker/10:0H-events_highpri
cpuhp/12
idle_inject/12
migration/12
ksoftirqd/12
kworker/12:0H-events_highpri
cpuhp/14
idle_inject/14
migration/14
ksoftirqd/14
kworker/14:0H-events_highpri
cpuhp/1
idle_inject/1
migration/1
ksoftirqd/1
kworker/1:0H-events_highpri
cpuhp/3
idle_inject/3
migration/3
ksoftirqd/3
kworker/3:0H-events_highpri
cpuhp/5
idle_inject/5
migr

Zoomed in on Brave by 20%.


Error: Invalid key sequence '--window'
Failure converting key sequence '--window' to keycodes
xdo_send_keysequence_window reported an error for string '--window'
X Error of failed request:  BadValue (integer parameter out of range for operation)
  Major opcode of failed request:  132 (XTEST)
  Minor opcode of failed request:  2 (X_XTestFakeInput)
  Value in failed request:  0x4
  Serial number of failed request:  39
  Current serial number in output stream:  41


In [ ]:
import webbrowser

# Open a webpage
url = "https://www.google.com"
webbrowser.get().open(url)

# Zoom in on the webpage


# Zoom out on the webpage
webbrowser.get().zoom(0.5)

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinterhtml import HtmlFrame
import requests
from bs4 import BeautifulSoup

def load_website():
    website_url = url_entry.get()
    try:
        # Fetch the HTML content of the website's homepage
        response = requests.get(website_url)
        homepage_html = response.text

        # Parse the homepage HTML
        soup = BeautifulSoup(homepage_html, 'html.parser')

        # Find all the <a> tags containing links to other pages
        links = soup.find_all('a', href=True)

        # Display the homepage HTML in the HtmlFrame
        html_frame.set_content(homepage_html)

        # Load and display HTML content of each linked page
        for link in links:
            page_url = link['href']
            page_response = requests.get(page_url)
            page_html = page_response.text
            html_frame.append_content(page_html)
    except Exception as e:
        print(f"Error loading website: {e}")

root = tk.Tk()
root.title("Website Renderer")

# URL Entry
url_label = ttk.Label(root, text="Enter Website URL:")
url_label.pack(pady=5)

url_entry = ttk.Entry(root, width=50)
url_entry.pack(pady=5)

# Load Button
load_button = ttk.Button(root, text="Load Website", command=load_website)
load_button.pack(pady=5)

# HtmlFrame to display website content
html_frame = HtmlFrame(root, horizontal_scrollbar="auto")
html_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)

root.mainloop()


In [38]:
import requests
from bs4 import BeautifulSoup

def render_website(url):
    # Fetch the HTML content of the website
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Print the title of the website
        print("Title:", soup.title.string)
        
        # Print other information or manipulate the HTML as needed
        # For example:
        # print(soup.find('p').text)  # Print the text of the first paragraph
        
    else:
        print("Failed to retrieve the website")

# Example usage
url = 'https://nhentai.to/g/359336/1'
render_website(url)


Title: nhentai : Free Hentai Manga, Doujinshi and Comics Online!


In [ ]:
import tkinter as tk
from tkinterhtml import HtmlFrame
import requests
from bs4 import BeautifulSoup

def render_website():
    url = url_entry.get()
    
    # Fetch the HTML content of the website
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the HTML content as a string
        html_content = str(soup)
        
        # Update the web view with the HTML content
        web_view.set_content(html_content)
        
    else:
        output_text.insert(tk.END, "Failed to retrieve the website")

# Create the GUI
root = tk.Tk()
root.title("Website Renderer")

# Create input field for URL
url_label = tk.Label(root, text="Enter URL:")
url_label.pack()
url_entry = tk.Entry(root, width=50)
url_entry.pack()

# Create button to render website
render_button = tk.Button(root, text="Render Website", command=render_website)
render_button.pack()

# Create web view to display website content
web_view = HtmlFrame(root, width=800, height=600)
web_view.pack()

# Run the GUI
root.mainloop()


In [8]:
from selenium import webdriver
driver  =  webdriver.Chrome()

In [9]:
driver.get('https://nhentai.to/g/359336/1')

In [73]:
driver.execute_script("document.body.style.zoom = '110%'")

In [77]:
driver.execute_script("window.scrollTo(0, 1000)")

In [56]:

# Path to the Chrome WebDriver executable


# URL of the website you want to render
url = "https://nhentai.to/g/359336/1"
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode, without opening a window
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration, can help with stability

# Create a Chrome WebDriver
driver = webdriver.Chrome( options=chrome_options)

# Navigate to the URL
driver.get(url)

# Wait for the page to fully render (you can adjust the wait time based on the website)
driver.implicitly_wait(10)  # Wait for a maximum of 10 seconds for elements to appear

# Get the rendered HTML content
html_content = driver.page_source

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Find all anchor tags
anchor_tags = soup.find_all("a")

# Extract and print the URLs
print("List of URLs on the page:")
for tag in anchor_tags:
    href = tag.get("href")
    if href:
        print(href)

# Close the WebDriver
driver.quit()


List of URLs on the page:
/
/random/
/tags/
/artists/
/characters/
/parodies/
/groups/
https://fapello.io/
/random/
/tags/
/artists/
/characters/
/parodies/
/groups/
/login/?next=/
/register/
/g/349954/2/
/g/349954/21/
/g/349954/2/
/g/349954/2/
/g/349954/21/
/g/359336/
/home/
mailto:adultxnetworkk@gmail.com
https://hentaihaven.xxx/
https://ehentai.to/
https://hentai2read.to/
https://hitomila.to/
https://myreadingmanga.to/
https://erome.io/
https://noodlemagazine.io/
https://hentaizilla.com/hentai-comics/
https://daftsex.bio/


In [1]:
import speech_recognition as sr

# Create a speech recognition object
r = sr.Recognizer()

# Use the default microphone as the audio source
with sr.Microphone() as source:
    # Listen for 5 seconds and recognize speech
    audio = r.listen(source, phrase_time_limit=5)

    try:
        # Convert the audio to text
        text = r.recognize_google(audio)
        print("You said: " + text)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand your audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

AttributeError: Could not find PyAudio; check installation